## Rule-based pipeline using SpaCy

In [1]:
# !python -m spacy download es_core_news_md

In [2]:
from utils import read_corpus, text_processor

In [3]:
corpus = read_corpus()

In [4]:
for k, v in corpus.items():
    print(f"{k}: {len(v)}")

A1: 94
A2: 62
B: 110
B1: 42


In [5]:
ex = corpus["A1"][4]["content"]
# print(ex)
# print("\n\n")
processed_ex = text_processor(ex)
print(processed_ex.text)
# print("\n\n")
# print(processed_ex.tokens)
# print("\n\n")
# print(processed_ex.lemmas)
# print("\n\n")
# print(processed_ex.tags)
# print("\n\n")
# print(processed_ex.parses)

caPítulo —¡Eh, Lucas!, ¡mira, allí! —¿Dónde? —Allí. ¿Ves a ese chico? —¿Quién? —Aquél, el de los vaqueros y las gafas. Allá, delante del supermercado. Lucas mira con atención al chico. Después dice: —No, ese chico no es. Es demasiado alto y no tiene el pelo negro. —¡Ah! —Además, tiene que entrar o salir de esta casa de ventanas verdes —Lucas señala una portería. —El número veintitrés. —Sí, el veintitrés. Lucas y Rony están en la terraza de un bar de una placita en el barrio de La Latina de Madrid. Están bebiendo una cerveza y miran con atención a la gente que pasa. Lucas es alto, de nariz grande y ojos pequeños. Rony es bajo, un poco gordo y lleva una chaqueta vaquera azul. � placita: diminutivo de plaza, «pequeña plaza». barrio de La Latina: barrio monumental de Madrid (cerca de la Plaza del Sol) que comprende parte del Madrid antiguo. Es un barrio muy acogedor en el que se encuentran muchos bares para ir de tapas. —Voy a tomar otra cerveza —dice—. ¿Sabes, Lucas? Con este... —¡Mira, R

In [6]:
def flatten(list_of_sents):
    return [tok for sent in list_of_sents for tok in sent]

In [22]:
from utils import frequency_list_50k, word_ranks, freq_lookup

In [23]:
def set_word_ranks(text, freq_list):
    """
    Given a tokenized text in the form of a list of lists of tokens, and a
    frequency list to search through, return a list of lists of integers,
    where the integers correspond to the ranks of the words in a frequency list.

    text: (list[list[str]]) the tokenized text
    freq_list: (list[str]) the ordered list of words to search through

    return: (list[list[int]]) the ranks of each of the tokens in the text
    """
    ranked_tokens = []
    for token in set(flatten(text)):
        ranked_tokens.append(freq_lookup(token, freq_list))
    return ranked_tokens

In [26]:
temp = [
    ("A1", 0),
    ("A1", 1),
    ("A1", 2),
    ("A1", 3),
    ("A1", 92),
    ("A2", 4),
    ("A2", 0),
    ("A2", -1),
    ("B1", 4),
    ("B1", 40),
    ("B1", 0),
]
freq_list = frequency_list_50k()
for level, i in temp:
    ex = corpus[level][i]["content"]
#     print(ex)
    p = text_processor(ex)
#     print(p.tokens)
    ranked_tokens = word_ranks(p.tokens, freq_list)
#     print(ranked_tokens)
    
    # Compute normalized frequency sum for each text
    flattened = flatten(ranked_tokens)
    print(f"{level} - {i}: {sum(flattened)/len(flattened):.2f}")

A1 - 0: 1823.22
A1 - 1: 1591.03
A1 - 2: 1163.57
A1 - 3: 2468.33
A1 - 92: 1188.82
A2 - 4: 1822.81
A2 - 0: 1821.08
A2 - -1: 1067.50
B1 - 4: 3062.71
B1 - 40: 2840.82
B1 - 0: 2704.01
